In [10]:
import json
from elasticsearch import Elasticsearch
from tqdm.auto import tqdm
import pandas as pd

In [2]:

with open('document_with_ids.json', 'rt') as f_in:
    documents = json.load(f_in)

In [3]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp',
 'id': 'c02e79ef'}

In [4]:
es_client = Elasticsearch("http://localhost:9200")

In [5]:
es_client.info()

ObjectApiResponse({'name': '030dd1857300', 'cluster_name': 'docker-cluster', 'cluster_uuid': 'dg2osB83QkSLHswpIDED-Q', 'version': {'number': '8.4.3', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '42f05b9372a9a4a470db3b52817899b99a76ee73', 'build_date': '2022-10-04T07:17:24.662462378Z', 'build_snapshot': False, 'lucene_version': '9.3.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'})

In [6]:

index_settings = {
    "settings": {
        "number_of_shards" : 1,
        "number_of_replicas" : 0
    },
    "mappings" : {
        "properties" : {
            "text" : {"type" : "text"},
            "section" : {"type" : "text"},
            "question" : {"type" : "text"},
            "course" : {"type" : "keyword"},
            "id" : {"type" : "keyword"},
        }
    }
}

index_name = "course_questions"
es_client.indices.delete(index = index_name, ignore_unavailable=True)
es_client.indices.create(index = index_name, body = index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course_questions'})

In [7]:
for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc)

100%|███████████████████████████████████████████████████████████████████████████████| 948/948 [00:03<00:00, 257.93it/s]


In [8]:
def elastic_search(query,course):
    search_query = {
        "size": 5,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^3", "text", "section"],
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": course
                    }
                }
            }
        }
    }

    response = es_client.search(index= index_name, body= search_query)  
    
    result_docs = []
    for hit in response['hits']['hits']:
        result_docs.append(hit['_source'])
        
    return result_docs

In [9]:
elastic_search(
    query="I just discovered the course. Can I still join?",
    course='data-engineering-zoomcamp'
)

[{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'section': 'General course-related questions',
  'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp',
  'id': '7842b56a'},
 {'text': 'You can start by installing and setting up all the dependencies and requirements:\nGoogle cloud account\nGoogle Cloud SDK\nPython 3 (installed with Anaconda)\nTerraform\nGit\nLook over the prerequisites and syllabus to see if you are comfortable with these subjects.',
  'section': 'General course-related questions',
  'question': 'Course - What can I do before the course starts?',
  'course': 'data-engineering-zoomcamp',
  'id': '63394d91'},
 {'text': 'Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it fin

In [11]:
df_ground_truth = pd.read_csv('ground_truth_data.csv')

In [14]:
ground_truth = df_ground_truth.to_dict(orient='records')

In [23]:
relevance_total = []

for q in tqdm(ground_truth):
    doc_id = q['document']
    results = elastic_search(query=q['question'], course=q['course'])
    relevance = [d['id'] == doc_id for d in results]
    relevance_total.append(relevance)

100%|█████████████████████████████████████████████████████████████████████████████| 4735/4735 [00:15<00:00, 297.23it/s]


In [27]:
example_metrix = [
    [False, False, False, False, False],
    [False, False, False, False, False],
    [False, False, False, False, False],
    [False, True, False, False, False],
    [False, False, False, False, False],
    [True, False, False, False, False],
    [False, False, False, True, False],
    [True, False, False, False, False],
    [True, False, False, False, False],
    [False, True, False, False, False],
    [False, False, False, False, True],
    [False, False, False, False, True]
]

In [29]:
def hit_rate(relavance_total):
    cnt = 0
    for line in relavance_total:
        if True in line:
            cnt = cnt + 1
    return cnt / len(relavance_total)

In [32]:
def mrr_score(relavance_total):
    score = 0.0
    for line in relavance_total:
        for rank in range(len(line)):
            if line[rank] == True:
                score = score + 1 / (rank + 1)
        
    return score / len(relavance_total)

In [31]:
hit_rate(relevance_total)

0.6378035902851109

In [34]:
mrr_score(relevance_total)

0.4952692713833152